# 9 Head poses Model Training

Neural Network model for a quick Head Pose estimation, using just facial points as input. The model was made for real-time control of a Pan-Tilt camera using face movements as the control signal. 

### Load dependencies, set random seed for reproducibility

In [97]:
import numpy as np
import pandas as pd
np.random.seed(14)

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import SGD

from keras.layers import Dropout, Activation, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization 
from keras import regularizers 
from keras.layers.convolutional import Conv1D

from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from numpy import array

### Read and preprocess the dataset

In [98]:
dfCenter = pd.read_csv('datasets/face_center.csv')
#dfCenter2 = pd.read_csv('datasets/face_center2.csv')
#dfCenter3 = pd.read_csv('datasets/face_center3.csv')
#dfCenter = dfCenter.append(dfCenter2, ignore_index = True).append(dfCenter3, ignore_index = True)

dfRight = pd.read_csv('datasets/face_right.csv')
#dfRight2 = pd.read_csv('datasets/face_right2.csv')
#dfRight3 = pd.read_csv('datasets/face_right3.csv')
#dfRight4 = pd.read_csv('datasets/face_right4.csv')
#dfRight = dfRight.append(dfRight2, ignore_index = True).append(dfRight3, ignore_index = True).append(dfRight4, ignore_index = True)

dfLeft = pd.read_csv('datasets/face_left.csv')
#dfLeft2 = pd.read_csv('datasets/face_left2.csv')
#dfLeft3 = pd.read_csv('datasets/face_left3.csv')
#dfLeft4 = pd.read_csv('datasets/face_left4.csv')
#dfLeft5 = pd.read_csv('datasets/face_left5.csv')
#dfLeft = dfLeft.append(dfLeft2, ignore_index = True).append(dfLeft3, ignore_index = True).append(dfLeft4, ignore_index = True).append(dfLeft5, ignore_index = True)

dfUp = pd.read_csv('datasets/face_up.csv')
dfDown = pd.read_csv('datasets/face_down.csv')
dfUpRight = pd.read_csv('datasets/face_up-right.csv')
dfUpLeft = pd.read_csv('datasets/face_up-left.csv')
dfDownRight = pd.read_csv('datasets/face_down-right.csv')

dfDownLeft = pd.read_csv('datasets/face_down-left.csv')
#dfDownLeft2 = pd.read_csv('datasets/face_down-left2.csv')
#dfDownLeft3 = pd.read_csv('datasets/face_down-left3.csv')
#dfDownLeft = dfDownLeft.append(dfDownLeft2, ignore_index = True).append(dfDownLeft3, ignore_index = True)

columns = list(dfCenter)

dfCenter['RESULT'] = 0
dfRight['RESULT'] = 1
dfLeft['RESULT'] = 2
dfUp['RESULT'] = 3
dfDown['RESULT'] = 4
dfUpRight['RESULT'] = 5
dfUpLeft['RESULT'] = 6
dfDownRight['RESULT'] = 7
dfDownLeft['RESULT'] = 8


df = dfCenter.append(dfRight, ignore_index=True).append(dfLeft, ignore_index = True)
df = df.append(dfUp, ignore_index = True).append(dfDown, ignore_index = True)
df = df.append(dfUpRight, ignore_index = True).append(dfUpLeft, ignore_index = True)
df = df.append(dfDownRight, ignore_index = True).append(dfDownLeft, ignore_index = True)


print(df.shape)

(4500, 61)


In [99]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,RESULT
4495,1.0,0.920769,0.897440,0.898442,0.871278,0.825369,0.782357,0.774431,0.801830,0.801830,...,0.324651,0.386135,0.455754,0.507020,0.480467,0.436155,0.374303,0.321068,0.265400,8
4496,1.0,0.907256,0.874157,0.860561,0.834543,0.783226,0.738433,0.735392,0.756263,0.768022,...,0.310963,0.362447,0.430281,0.508652,0.476326,0.413510,0.356212,0.307531,0.248106,8
4497,1.0,0.906459,0.865125,0.851669,0.825920,0.775133,0.730803,0.734114,0.745201,0.760087,...,0.328798,0.377528,0.434693,0.490143,0.477007,0.427999,0.372601,0.319534,0.250663,8
4498,1.0,0.909009,0.874791,0.869813,0.832541,0.786036,0.757910,0.736437,0.758546,0.776774,...,0.320208,0.384563,0.451509,0.517020,0.493626,0.442059,0.380781,0.326180,0.260370,8
4499,1.0,0.902666,0.876755,0.866617,0.835005,0.780461,0.747263,0.744515,0.774200,0.790083,...,0.362850,0.418881,0.486761,0.522545,0.488600,0.437134,0.369495,0.307855,0.233031,8


### Define input data and desired output, and then split training and test observations

In [100]:
X = df[columns]
y = df['RESULT']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=14)

c = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')

n_classes = 9
#one hot encoding
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

print (y_test[:5])

[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [92]:
X = []
for i in range(1,X_train.shape[0]+1):
    X.append(X_train[0:i].tolist()[0])
len(X)

Y = []
for i in range(1,y_train.shape[0]+1):
    Y.append(y_train[0:i].tolist()[0])
len(Y)


3600

In [114]:
X = np.array(X_train).astype('float32')
Y = np.array(y_train)
len(X)

3600

### Define Neural Network architecture

In [109]:
def create_dense(layer_sizes):
    model = Sequential()

# model.add(Conv1D(64, kernel_size = 3, activation='relu', input_shape=(60,1)))
# model.add(Flatten())
#     model.add(Dense(layer_sizes[0], activation='relu', input_shape=(60,)))
    
#     model.add(Conv2D(32, (8,8),strides = (4,4), padding = 'same', input_shape = (60,1,1), activation = 'relu'))
    model.add(Conv1D(100, 10, activation='relu', input_shape=(60,1)))

#     model.add(Activation('relu'))

    for s in layer_sizes[1:]:
        model.add(Dense(units = s, activation = 'relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
    #model.add(Dense(128, activation='relu', input_shape=(60,)))
    #model.add(BatchNormalization())

    #model.add(Dense(256, activation='relu'))
    #model.add(BatchNormalization())
    #model.add(Dropout(0.5))

    #model.add(Dense(128, activation='relu'))
    #model.add(Dropout(0.5))
    
#     model.add(Conv1D(100, 10,activation='relu'))

    model.add(Dense(9, activation='softmax'))
    
    return model

# model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_test, y_test))
def evaluate(model, batch_size=128, epochs=20):
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.00125), metrics=['accuracy'])
    
    X = np.array(X_train).astype('float32')
    Y = np.array(y_train).astype('float32')
    X = X.reshape(128,1,60)
    Y = Y.reshape(128,1,60)
    history = model.fit(X, Y, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
#     X_new = X_train.tolist()
#     Y_new = y_train.tolist()
    
    
#     model.train_on_batch(X_train, y_train)
    loss, accuracy  = model.evaluate(X_test, y_test, verbose=False)

# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['training', 'validation'], loc='best')
# plt.show()

    print(f'Test loss: {loss:.3}')
    print(f'Test accuracy: {accuracy:.3}')

# for layers in range(1, 5):
model = create_dense([32] * 5)
evaluate(model)

# X_new = array([[1.0,0.8634537263852682,0.7550683797983376,0.6766083980311693,0.6105539851678169,0.5704414015790741,0.5753162861652857,0.6279255072837533,0.634317188327362,0.6149428731822305,0.5660729283392674,0.5491935638206207,0.5535083874348202,0.6083476675122917,0.6882788400699552,0.7915827354996343,0.9179179909700335,0.8650096079496821,0.8086468979375749,0.695752227878763,0.5959801903225727,0.4942783927093914,0.5101372039962554,0.6191336186419911,0.7278660554122794,0.8342086149396643,0.880534685421328,0.3621523485241818,0.19189191551595036,0.04548274598178804,0.15016195917834194,0.19350222555566068,0.18193098392715215,0.2176528799919824,0.22741372990894018,0.24745289125103817,0.6512138184336927,0.6156153095484503,0.5280701549410947,0.4118637820051627,0.4516324456341951,0.5495701107032686,0.4269083321253682,0.5551876683829843,0.6408065356530825,0.6761496285820855,0.5753162861652857,0.4781099874671581,0.35726400214981907,0.30340775880150395,0.27926554184691915,0.30238330649415557,0.28657739482928707,0.3232156388438138,0.37643670446703803,0.40604656199615247,0.43743904356956603,0.4314872146239643,0.4259380846616632,0.39885023003525083

# ]])
# Y_new = model.predict_classes(X_new)
# print("X=%s, Predicted=%s" % (X_new[0], Y_new[0]))





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_15 (Conv1D)           (None, 51, 100)           1100      
_________________________________________________________________
dense_248 (Dense)            (None, 51, 32)            3232      
_________________________________________________________________
batch_normalization_111 (Bat (None, 51, 32)            128       
_________________________________________________________________
dropout_75 (Dropout)         (None, 51, 32)            0         
_________________________________________________________________
dense_249 (Dense)            (None, 51, 32)            1056      
_________________________________________________________________
batch_normalization_112 (Bat (None, 51, 32)            128       
_________________________________________________________________
dropout_76 (Dropout)         (None, 51, 32)            0         
__________

ValueError: cannot reshape array of size 216000 into shape (128,1,60)

In [37]:
# model.summary()

### Configure the model

In [38]:
# model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.00125), metrics=['accuracy'])

### Train the model

In [40]:
# model.fit(X_train, y_train, batch_size=32, epochs=100, verbose=1, validation_data=(X_test, y_test))
def evaluate(model, batch_size=128, epochs=5):
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.00125), metrics=['accuracy'])
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))

    loss, accuracy  = model.evaluate(X_test, y_test, verbose=False)

# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['training', 'validation'], loc='best')
# plt.show()

    print(f'Test loss: {loss:.3}')
    print(f'Test accuracy: {accuracy:.3}')


### Saving the model

In [41]:
for layers in range(1, 5):
    model = create_dense([32] * layers)
    evaluate(model)

# model.save('model/model_9_positions.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 51, 100)           1100      
_________________________________________________________________
dense_182 (Dense)            (None, 51, 9)             909       
Total params: 2,009
Trainable params: 2,009
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking input: expected conv1d_3_input to have 3 dimensions, but got array with shape (3600, 60)